In [1]:
import numpy as np
# from cdlib import algorithms, viz, evaluation
import networkx as nx
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import entropy

In [2]:
with open('../pkl/COVID_sorted_W_tensors_ver2.pkl', 'rb') as f:
    travel_tensors = pickle.load(f) 
    
# with open('../pkl/global_serial_graphs.pkl', 'rb') as f:
#     serial_graphs = pickle.load(f) 

In [3]:
travel_tensors.shape

(22, 54, 108)

In [4]:
home_df = pd.read_csv('../COVIDdata/PatientInfo.csv')

In [5]:
condition = (home_df['province']=="Seoul") | (home_df['province']=="Gyeonggi-do")
home_df = home_df[condition]

In [6]:
home_df.describe(include= 'all')

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
count,2.520000e+03,1414,1414,2520,2520,2520,2434,774,163,382,2520,115,1,2520
unique,NaN,2,10,8,2,57,35,334,37,113,145,46,1,3
top,NaN,female,20s,Korea,Seoul,Seongnam-si,contact with patient,2000000205,2,2020-03-18,2020-03-10,2020-03-04,2020-02-25,isolated
freq,NaN,743,353,2505,1312,173,948,51,27,11,56,10,1,1466
mean,1.479167e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,4.995656e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.000001e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.000001e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.000001e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
home_df = home_df[['patient_id', 'city']]

In [8]:
home_df.columns = ['patient_id', 'home']

In [9]:
path_df = pd.read_csv("../COVIDdata/PatientRoute.csv")

In [10]:
condition = (path_df['province']=="Seoul") | (path_df['province']=="Gyeonggi-do")
df = path_df[condition]

In [11]:
merged_df = pd.merge(df, home_df[['patient_id','home']], how='inner',left_on=['patient_id'],right_on=['patient_id'])

In [12]:
merged_df['date']= pd.to_datetime(merged_df['date'])

In [13]:
merged_df

,patient_id,date,province,city,type,latitude,longitude,home
0,1000000001,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.61525,126.7156,Gangseo-gu
1,1000000001,2020-01-24,Seoul,Jung-gu,hospital,37.56724,127.0057,Gangseo-gu
2,1000000002,2020-01-25,Seoul,Seongbuk-gu,etc,37.59256,127.0170,Jungnang-gu
3,1000000002,2020-01-26,Seoul,Seongbuk-gu,store,37.59181,127.0168,Jungnang-gu
4,1000000002,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.56399,127.0295,Jungnang-gu
...,...,...,...,...,...,...,...,...
5377,2000000673,2020-04-26,Gyeonggi-do,Yongin-si,hospital,37.32250,127.0977,Yongin-si
5378,2000000673,2020-04-27,Gyeonggi-do,Anseong-si,hospital,37.01722,127.2604,Yongin-si
5379,2000000674,2020-04-26,Gyeonggi-do,Yongin-si,hospital,37.32250,127.0977,Yongin-si
5380,2000000674,2020-04-27,Gyeonggi-do,Anseong-si,hospital,37.00797,127.2797,Yongin-si


In [14]:
# merged_df.to_csv('COVID_merged.csv')

In [15]:
travel_df = pd.read_csv("../csv/COVID_travel_df.csv")
nodes = travel_df['city'].unique().T
nodes
label_dict = dict(zip(nodes, range(54)))
# label_dict

In [16]:
# def save_object(obj, filename):
#     with open(filename, 'wb') as output:  # Overwrites any existing file.
#         pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# save_object(label_dict, '../pkl/COVID_label_dict.pkl')


In [17]:
country_num = len(nodes)
week_slices = pd.date_range('2020-01-19', periods=23, freq='W')

home_adj_list = []

for i in range(len(week_slices)-1):
    print(i)
    home_adj = np.zeros([country_num,country_num])
    start = week_slices[i]
    end = week_slices[1+i]
    df_intime = merged_df[(merged_df['date']>=start) & (merged_df['date'] < end)]
#     df_intime = df_intime[(df_intime['city']!=df_intime['home'])]
    
    for index, row in df_intime.iterrows():
#         print(index, row['home'])
        if(row['home']=='etc'):
            continue
        r = label_dict[row['home']]
        c = label_dict[row['city']]
        if(r!=c):
            home_adj[r,c] +=1
    home_adj_list.append(home_adj)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [18]:
home_adj = np.array(home_adj_list)

In [19]:
np.sum(home_adj[:,19,:],axis=1)

array([ 0.,  0.,  0.,  0.,  3.,  0., 10.,  7.,  2.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  2.,  1.,  0.,  8.,  3.,  1.])

In [20]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

save_object(home_adj_list, '../pkl/COVID_home_adj_ver2.pkl')

In [21]:
with open('../pkl/new_COVID_order_ver2.pkl', 'rb') as f:
    new_order = pickle.load(f) 

In [22]:
import copy
new_tensor = copy.deepcopy(home_adj)

In [23]:
for i in range(len(home_adj)):
    adjmat = home_adj[i][:, new_order][new_order,:]
    new_tensor[i] = adjmat

In [24]:
save_object(new_tensor, '../pkl/COVID_sorted_home_weekly_ver2.pkl')